## Notice:
Uncomment the next cell and run once if you have never 
set your Colab notebook work environment to do C, C++ and NVCC, GCC compiler.

# Prepare the Environment

!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2
!nvcc --version

! apt-get install gcc-7 g++-7
!which gcc-7
! update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-7 70 --slave /usr/bin/g++ g++ /usr/bin/g++-7
!gcc --version

# START HERE

In [ ]:
# check if there is GPU in the device
!nvidia-smi

In [ ]:
# always run this cell before using NVCC commands
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-oku4mr80
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-oku4mr80
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4304 sha256=a56c3e7f6664e8a60d21e0c58acbc628a4ff8fe00d39511061aa69ef58c0dacc
  Stored in directory: /tmp/pip-ephem-wheel-cache-rob7g0oo/wheels/f3/08/cc/e2b5b0e1c92df07dbb50a6f024a68ce090f5e7b2316b41756d
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [ ]:
# A quick check if everything working fine:
# if the output is 2 + 3 = 5 it means we can work with the GPU now

%%cu  #Tells the compiler the following code is CUDA-C Program
#include <iostream>
__global__ void add(int a, int b, int *c) {
  *c = a + b;
}

int main() {
  int a = 2, b = 3, c;
  int *dev_c;
  cudaMalloc(&dev_c, sizeof(int));
  add<<<1, 1>>>(a, b, dev_c);
  cudaMemcpy(&c, dev_c, sizeof(int), cudaMemcpyDeviceToHost);
  printf("%d + %d = %d\n", a, b, c);
  cudaFree(dev_c);
  return 0;
}

# This is where the real execution of the project starts.

In [ ]:
# Go to the directory where you have saved the files downloaded from github.
# Example: my files are in "/content/drive/MyDrive/CUDA"
%cd /content/drive/MyDrive/CUDA

# to check the current working directory
!pwd

# sometimes the sbatch file have read only permission the following line will change the permistion to executable.
!chmod +x Merge.sbatch

/content/drive/MyDrive/CUDA


In [ ]:
# you can run the .cu file as follows:
!./Merge_sort_1 1000 10 "false" "ture"

# Or run the sbatch file as follows
!./Merge.sbatch

Merge_Sort By Sami Osman
Array size:          1000
Block size:          10
Analysis flag        false
Shared memory flag:  false
Saving unsorted array to Unsorted.txt file ...
@@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@ MergeSortWithCuda @@@@
@@@@@@@@@@@@@@@@@@@@@@@@@@@@
First loop Configuration
Block/Grid: 10 
Thread/Block: 51 
Total threads: 510 
T_Time: 0.000652
@@@@@@@@@@@@@@@@@@@@@@@@
@@ SORTING IS CORRECT @@
@@@@@@@@@@@@@@@@@@@@@@@@
Saving sorted array to sorted.txt file ...
